In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import colors
import scipy.integrate as integrate
from matplotlib.patches import Circle
import matplotlib.cbook as cbook
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import Normalize
import matplotlib.ticker as ticker
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl
from matplotlib.text import Annotation
from mpl_toolkits.mplot3d.proj3d import proj_transform
from matplotlib.patches import FancyArrowPatch

from mpl_toolkits.mplot3d import proj3d
from matplotlib.colors import Normalize
import matplotlib.ticker as ticker

import matplotlib.patches as patches

import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats

import matplotlib.cm as cm
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

In [ ]:
def getImage(path, zoom=1):
    return OffsetImage(plt.imread(path), zoom=zoom)


def interpol_values(df,k):
    interpolated_values= np.interp(np.linspace(-0.015,0.015,NPOSES), df.iloc[k], df.iloc[k+1] )
    #plt.plot(interpolated_values)
    return(interpolated_values)

NTIMES=101
NPOSES=202

def resample_2D_veloc(file): #"Simus_vitesses_2D/full_R07_v40.csv"
    df=pd.read_csv(file, sep=',', encoding='utf-8',header = 0)
    df.drop([k*3 for k in range(NTIMES)], axis=0, inplace=True)
    df.reset_index(inplace = True, drop=True)

    new_df= np.zeros((NTIMES,NPOSES)) #time, x_vals
    for k in range(2*NTIMES):
        if k%2==0:
            new_df[int(k/2),:]=interpol_values(df,k)
    return(new_df)

df_R07v05=resample_2D_veloc("Flow_Velocities_2D/full_R07_v05.csv")
df_R07v10=resample_2D_veloc("Flow_Velocities_2D/full_R07_v10.csv")
df_R07v20=resample_2D_veloc("Flow_Velocities_2D/full_R07_v20.csv")
df_R07v40=resample_2D_veloc("Flow_Velocities_2D/full_R07_v40.csv")

df_R15v05=resample_2D_veloc("Flow_Velocities_2D/full_R15_v05.csv")
df_R15v10=resample_2D_veloc("Flow_Velocities_2D/full_R15_v10.csv")
df_R15v20=resample_2D_veloc("Flow_Velocities_2D/full_R15_v20.csv")
df_R15v40=resample_2D_veloc("Flow_Velocities_2D/full_R15_v40.csv")

df_R25v05=resample_2D_veloc("Flow_Velocities_2D/full_R25_v05.csv")
df_R25v10=resample_2D_veloc("Flow_Velocities_2D/full_R25_v10.csv")
df_R25v20=resample_2D_veloc("Flow_Velocities_2D/full_R25_v20.csv")
df_R25v40=resample_2D_veloc("Flow_Velocities_2D/full_R25_v40.csv")

Vitesses_dico ={}
Vitesses_dico["R7.5_v5"]  =df_R07v05
Vitesses_dico["R7.5_v10"] =df_R07v10
Vitesses_dico["R7.5_v20"] =df_R07v20
Vitesses_dico["R7.5_v40"] =df_R07v40
Vitesses_dico["R15_v5"] =df_R15v05
Vitesses_dico["R15_v10"]=df_R15v10
Vitesses_dico["R15_v20"]=df_R15v20
Vitesses_dico["R15_v40"]=df_R15v40
Vitesses_dico["R25_v5"] =df_R25v05
Vitesses_dico["R25_v10"]=df_R25v10
Vitesses_dico["R25_v20"]=df_R25v20
Vitesses_dico["R25_v40"]=df_R25v40

masse=1.204*0.01*0.01*0.008
Vitesses_dico["P_R7.5_v5"]  =pd.read_csv("Momentums_3D/2_Volume_R7.5_V5.csv",header=None)*masse
Vitesses_dico["P_R7.5_v10"] =pd.read_csv("Momentums_3D/2_Volume_R7.5_V10.csv",header=None)*masse
Vitesses_dico["P_R7.5_v20"] =pd.read_csv("Momentums_3D/2_Volume_R7.5_V20.csv",header=None)*masse
Vitesses_dico["P_R7.5_v40"] =pd.read_csv("Momentums_3D/2_Volume_R7.5_V40.csv",header=None)*masse
Vitesses_dico["P_R15_v5"] =pd.read_csv("Momentums_3D/2_Volume_R15_V5.csv",header=None)*masse
Vitesses_dico["P_R15_v10"]=pd.read_csv("Momentums_3D/2_Volume_R15_V10.csv",header=None)*masse
Vitesses_dico["P_R15_v20"]=pd.read_csv("Momentums_3D/2_Volume_R15_V20.csv",header=None)*masse
Vitesses_dico["P_R15_v40"]=pd.read_csv("Momentums_3D/2_Volume_R15_V40.csv",header=None)*masse
Vitesses_dico["P_R25_v5"] =pd.read_csv("Momentums_3D/2_Volume_R25_V5.csv",header=None)*masse
Vitesses_dico["P_R25_v10"]=pd.read_csv("Momentums_3D/2_Volume_R25_V10.csv",header=None)*masse
Vitesses_dico["P_R25_v20"]=pd.read_csv("Momentums_3D/2_Volume_R25_v20.csv",header=None)*masse
Vitesses_dico["P_R25_v40"]=pd.read_csv("Momentums_3D/2_Volume_R25_v40.csv",header=None)*masse



def extract_momentum(v,r):
    NPOSES=202
    NTIMES=101
    if r != 7.5:
        r=int(r)
    Pstring= "P_R"+ str(r)+"_v"+str(int(v))
    #Vstring= "V_R"+ str(r)+"_v"+str(int(v))
    
    TP=Vitesses_dico[Pstring]
    #TV=Vitesses_dico[Vstring]
    
    X0=np.linspace(0,6/v,NTIMES) 
    X=X0-4/v
    idx=np.min(np.argwhere(X > 0.005))
    #print(idx)
    return(X[:idx],TP[0].iloc[:idx]) #,TV[NPOSES//2].iloc[:idx] 





def format_axes(ax,xlabel,ylabel, reffig, loclegend):
    ax.text(reffig[0], reffig[1], reffig[2], fontsize=reffig[3], fontweight="bold", transform=ax.transAxes)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.set_xlabel(xlabel, fontsize=8)
    ax.set_ylabel(ylabel, fontsize=8)
    ax.tick_params(axis='x', labelsize=8)
    ax.tick_params(axis='y', labelsize=8)
    ax.legend(fontsize=8,loc=loclegend, frameon=False)    
    
    

def plot_comp_rayons(keys,v, ax, letter, lst):
    X0=np.linspace(0,6/v,NTIMES)
    X=X0 - 4/v
    idx=np.min(np.argwhere(X > 0.005))
    ax.plot(X[:idx],Vitesses_dico[keys[0]][:idx,NPOSES//2],color='grey',linestyle=linestyles[1],alpha=0.25)
    ax.plot(X[:idx],Vitesses_dico[keys[1]][:idx,NPOSES//2],color='grey',linestyle=linestyles[1],alpha=0.25)
    ax.plot(X[:idx],Vitesses_dico[keys[2]][:idx,NPOSES//2],color='grey',linestyle=linestyles[1],alpha=0.25)
    
    ax.plot(X[:idx],Vitesses_dico[keys[0]][:idx,NPOSES//2],color='tab:blue',linestyle=linestyles[lst],label="D=7.5mm")
    ax.plot(X[:idx],Vitesses_dico[keys[1]][:idx,NPOSES//2],color='tab:orange',linestyle=linestyles[lst],label="D=15mm")
    ax.plot(X[:idx],Vitesses_dico[keys[2]][:idx,NPOSES//2],color='tab:green',linestyle=linestyles[lst],label="D=25mm")
    format_axes(ax,'Time (s)',"r$\Phi(t)$ (m/s) at X=0m", [0.02,0.95,letter,10], "upper center")

    
def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw=None, cbarlabel="", **kwargs):

    if ax is None:
        ax = plt.gca()

    if cbar_kw is None:
        cbar_kw = {}

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # Show all ticks and label them with the respective list entries.
    ax.set_xticks(range(data.shape[1]), labels=col_labels,
                  rotation=-30, ha="right", rotation_mode="anchor")
    ax.set_yticks(range(data.shape[0]), labels=row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Turn spines off and create white grid.
    ax.spines[:].set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=("black", "white"),
                     threshold=None, **textkw):

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = mpl.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

def get_cube(limits=None):

    v = np.array([[0, 0, 0], [0, 0, 1],
                  [0, 1, 0], [0, 1, 1],
                  [1, 0, 0], [1, 0, 1],
                  [1, 1, 0], [1, 1, 1]], dtype=int)

    if limits is not None:
        v = limits[np.arange(3)[np.newaxis, :].repeat(8, axis=0), v]

    e = np.array([[0, 1], [0, 2], [0, 4],
                  [1, 3], [1, 5],
                  [2, 3], [2, 6],
                  [3, 7],
                  [4, 5], [4, 6],
                  [5, 7],
                  [6, 7]], dtype=int)

    f = np.array([[0, 2, 3, 1],
                  [0, 4, 5, 1],
                  [0, 4, 6, 2],
                  [1, 5, 7, 3],
                  [2, 6, 7, 3],
                  [4, 6, 7, 5]], dtype=int)

    return v, e, f


class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        super().__init__((0,0), (0,0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def do_3d_projection(self, renderer=None):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, self.axes.M)
        self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))

        return np.min(zs)



def draw_all_curves(ax):
    for v in [5,10,20,40]:
        for r,color in zip([7.5,15,25],['tab:blue','tab:orange','tab:green']):
            T,V=extract_momentum(v,r)
            ax.plot(v,T,V*1e14, color=color)
    
    
    #ax.add_collection3d(plt.fill_between(x, 0.95*z, 1.05*z, color='r', alpha=0.3), zs=1, zdir='y')
    #ax.add_collection3d(plt.fill_between(x, 0.90*z, 1.10*z, color='g', alpha=0.3), zs=2, zdir='y')
    #ax.add_collection3d(plt.fill_between(x, 0.85*z, 1.15*z, color='b', alpha=0.3), zs=3, zdir='y')
    
    ax.set_xlabel('Velocity (cm/s)')
    ax.set_zlabel('Momentum (N.s)')
    ax.set_ylabel('Time to collision (s)')
    
    
    ax.xaxis.set_major_locator(ticker.FixedLocator(([5,10,20,40])))
    ax.xaxis.set_major_formatter(ticker.FixedFormatter((["5","10","20","40"])))
    
    

    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    
    
    MAX=1+int(1e14*np.nanmax(extract_momentum(5,25)[1]))
    yy, zz = np.meshgrid((-1,0), range(0,MAX+1))
    xx = yy*0+5
    ax.plot_surface(xx, yy, zz,alpha=0.05,color='tab:red')
    
    MAX=4 #int(1e14*np.nanmax(extract_momentum(10,25)[1]))
    yy, zz = np.meshgrid((-1,0), range(0,MAX+1))
    xx = yy*0+10
    ax.plot_surface(xx, yy, zz,alpha=0.05,color='tab:green')
    
    MAX=8#int(1e14*np.nanmax(extract_momentum(20,25)[1]))
    yy, zz = np.meshgrid((-1,0), range(0,MAX+1))
    xx = yy*0+20
    ax.plot_surface(xx, yy, zz,alpha=0.05,color='tab:orange')
    
    MAX=14
    yy, zz = np.meshgrid((-1,0), range(0,MAX+1))
    xx = yy*0+40
    ax.plot_surface(xx, yy, zz,alpha=0.05,color='tab:blue')
    
    ax.invert_xaxis()
    tmp_planes = ax.zaxis._PLANES 
    ax.zaxis._PLANES = ( tmp_planes[2], tmp_planes[3], 
                     tmp_planes[0], tmp_planes[1], 
                     tmp_planes[4], tmp_planes[5])
    ax.view_init(10, -30)




def imshow3d(ax, array, value_direction='z', pos=0, norm=None, cmap=None,annotatecolorbar=False):

    if norm is None:
        norm = Normalize()
        colors = plt.get_cmap(cmap)(norm(array))
    else:
        colors = plt.get_cmap(cmap)(array/np.max(maxMomentum))

    if value_direction == 'x':
        nz, ny = array.shape
        zi, yi = np.mgrid[0:nz + 1, 0:ny + 1]
        xi = np.full_like(yi, pos)
    elif value_direction == 'y':
        nx, nz = array.shape
        xi, zi = np.mgrid[0:nx + 1, 0:nz + 1]
        yi = np.full_like(zi, pos)
    elif value_direction == 'z':
        ny, nx = array.shape
        yi, xi = np.mgrid[0:ny + 1, 0:nx + 1]
        zi = np.full_like(xi, pos)
    else:
        raise ValueError(f"Invalid value_direction: {value_direction!r}")
    ax.plot_surface(xi, yi, zi, rstride=1, cstride=1, facecolors=colors, shade=False)

    if annotatecolorbar==True:
        m = cm.ScalarMappable(cmap=cm.jet)
        m.set_array(array)
        ax.colorbar(m)


In [ ]:
radius=[7.5, 15, 25]
velocities=[5,10,20,40]

"""
for v in velocities:
    for r in radius:
        T,V=extract_momentum(v,r)
"""        

FULLDF=pd.DataFrame(columns=('D','v','Time','Momentum'))
SmallDF=pd.DataFrame(columns=('D','v','MaxMomentum'))

for velo in velocities:
    for diam in radius:
        T,V=extract_momentum(velo,diam)
        
        N=np.asarray((np.repeat(diam,len(T)),np.repeat(velo,len(T)),T,V)).T#.reshape(-1,2)
        DF=pd.DataFrame(data=N, columns=('D','v','Time','Momentum'))

        sDF=pd.DataFrame(data=np.asarray((diam,velo,np.nanmax(V))).reshape(1,3), columns=('D','v','MaxMomentum'))

        
        
        FULLDF = pd.concat((FULLDF, DF), axis=0)#.reset_index()
        SmallDF= pd.concat((SmallDF, sDF), axis=0)
FULLDF.reset_index()
SmallDF.reset_index()
FULLDF.to_csv('Regression_Momentum.csv')
SmallDF.to_csv('Regression_MaxMomentum.csv')

In [ ]:
pp=PdfPages('Poster_figure_vitesse.pdf')

fig,axOFF = plt.subplots(figsize=(12,12))
axOFF.axis('off')

#ax=plt.subplot2grid((4,4),(0,0), colspan=2, rowspan=2, projection='3d')
#ax99=plt.subplot2grid((4,4),(0,2), colspan=2, rowspan=2)
#ax1=plt.subplot2grid((4,4),(2,0), rowspan=2)
#ax2=plt.subplot2grid((4,4),(2,2), rowspan=2)

ax=plt.subplot2grid((3,3),(0,0), projection='3d')
ax99=plt.subplot2grid((3,3),(1,0))
ax1=plt.subplot2grid((3,3),(0,1))
ax2=plt.subplot2grid((3,3),(0,2))

ax4=plt.subplot2grid((3,3),(1,1), colspan=2,projection='3d')


ax3=plt.subplot2grid((3,3),(2,1))

ax5=plt.subplot2grid((3,3),(2,0),projection='3d')


ax6=plt.subplot2grid((3,3),(2,2))




limits = np.array([[-1, 1],
                   [-1, 1],
                   [-1, 1]])

v, e, f = get_cube(limits)


for i, j in e:
    ax.plot(*v[[i, j], :].T, color='black', ls='-')

r = [-1,1]
X, Y = np.meshgrid(r, r)
z = np.array([[1, 1]])
ax.plot_surface(X,Y, 1*z, alpha=0.25,color='lightblue')
ax.plot_surface(X,Y,-1*z, alpha=0.25,color='lightblue')
ax.plot_surface(X,-1*z,Y, alpha=0.25,color='lime')
ax.plot_surface(X,1*z,Y, alpha=0.25,color='lime')
ax.plot_surface(1*z,X,Y, alpha=0.25,color='lightblue')
ax.plot_surface(-1*z,X,Y, alpha=0.25,color='lightblue')
ax.view_init(10, -20)
ax.grid(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_zticks([])
ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.xaxis.line.set_color((1.0, 1.0, 1.0, 0.0))
ax.yaxis.line.set_color((1.0, 1.0, 1.0, 0.0))
ax.zaxis.line.set_color((1.0, 1.0, 1.0, 0.0))
arrow_prop_dict = dict(mutation_scale=20, arrowstyle='->', color='cadetblue', shrinkA=0, shrinkB=0)
a = Arrow3D([0, 0], [1.5, 1], [0, 0], **arrow_prop_dict)
ax.add_artist(a)
a = Arrow3D([0, 0], [-1.5, -1], [0, 0], **arrow_prop_dict)
ax.add_artist(a)
a = Arrow3D([0, 0], [0, 0], [1.5, 1], **arrow_prop_dict)
ax.add_artist(a)
ax.text(0, -1.75, 0, "input", color='limegreen',fontsize=8)
ax.text(0, -1.75, -0.15, "u=0", color='limegreen',fontsize=8)
ax.text(0, -0.5, 1.6, "open boundaries (x4)", color='cadetblue',fontsize=8)
ax.text(0, 1.55, 0, "output", color='limegreen',fontsize=8)
ax.text(0, 1.55, -0.15, "P=0", color='limegreen',fontsize=8)


arrow_prop_dict_double = dict(mutation_scale=15, arrowstyle='<->', color='grey', shrinkA=0, shrinkB=0)

a = Arrow3D([1, 1], [-0.95, -0.95], [-1, 1], **arrow_prop_dict_double)
ax.add_artist(a)
ax.text(1, -0.9, 0, "10 cm", 'z',color='grey',fontsize=8)

a = Arrow3D([1.2, 1.2], [-1, 1], [-1, -1], **arrow_prop_dict_double)
ax.add_artist(a)
ax.text(1.1, -0.2, -1.2, "20 cm", color='grey',fontsize=8)

theta = np.linspace(0, 2 * np.pi, 10)
phi = np.linspace(0, np.pi, 50)
theta, phi = np.meshgrid(theta, phi)
r = 0.2
x = r * np.sin(phi) * np.cos(theta)
y = r * np.sin(phi) * np.sin(theta)
z = r * np.cos(phi)
ax.plot_surface(x, y, z, color='grey', alpha=0.8)

arrow_prop_dict_bold = dict(mutation_scale=30, arrowstyle='-|>', color='black', shrinkA=0, shrinkB=0)
a = Arrow3D([0, 0], [0, 0.5], [0, 0], **arrow_prop_dict_bold)
ax.add_artist(a)

ax.set_xlim(-1.1,1.1)
ax.set_ylim(-1.1,1.1)
ax.set_zlim(-1.1,1.1)

#############################################################################
#############################################################################

fts=8
circle1=Circle([0-0.75/2,0], 0.75/2 ,color='tab:blue',fill=False, clip_on = False, alpha=1)
circle2=Circle([0- 1.5/2,0], 1.5/2 ,color='tab:orange',fill=False, clip_on = False, alpha=1)
circle3=Circle([0-2.5/2,0], 2.5/2 ,color='tab:green',fill=False, clip_on = False, alpha=1)
ax99.add_patch(circle1)       
ax99.add_patch(circle2)   
ax99.add_patch(circle3)  

#Volume
ax99.vlines(4,-0.5,0.5, linestyle='solid', color='tab:pink')
ax99.vlines(5,-0.5,0.5, linestyle='solid', color='tab:pink')
ax99.hlines(0.5,4,5, linestyle='solid', color='tab:pink')
ax99.hlines(-0.5,4,5, linestyle='solid', color='tab:pink')


ax99.set_xlim([-3,5])
ax99.set_ylim([-2,4])
ax99.set_aspect("equal", adjustable="datalim")

ax99.vlines(0,0,-1.8,linestyle='dotted',color='grey')
ax99.vlines(4,-0.5,-1.8,linestyle='dotted',color='grey')
ax99.annotate(text='', xy=(0,-1.8), xytext=(4,-1.8), arrowprops=dict(arrowstyle='<->',color='grey'))
ax99.text(1.5,-1.7,'z=4cm',color='grey',fontsize=fts)

ax99.annotate(text='', xy=(0,0), xytext=(1,0), arrowprops=dict(arrowstyle='<-',color='black'))
ax99.text(0.6,0.1,'v',color='black',fontsize=fts)

#ax99.annotate(text='', xy=(-2.6,1.5), xytext=(0,1.5), arrowprops=dict(arrowstyle='|-|',color='red'))
ax99.text(-1.4,1.6,'D',color='black',fontsize=fts)
a=0.4

ax99.annotate(text='', xy=(2,3.5), xytext=(3,3.5), arrowprops=dict(arrowstyle='<-',color='black', linestyle='dashed'))
ax99.annotate(text='', xy=(2,3.5-a), xytext=(3,3.5-a), arrowprops=dict(arrowstyle='<-',color='black', linestyle='solid'))
ax99.annotate(text='', xy=(2,3.5-2*a), xytext=(3,3.5-2*a), arrowprops=dict(arrowstyle='<-',color='black', linestyle='dotted'))
ax99.annotate(text='', xy=(2,3.5-3*a), xytext=(3,3.5-3*a), arrowprops=dict(arrowstyle='<-',color='black', linestyle=(0, (3, 5, 1, 5))))


ax99.text(3.1, 3.5, 'v=40 cm/s',fontsize=fts)
ax99.text(3.1, 3.5-a, 'v=20 cm/s',fontsize=fts)
ax99.text(3.1, 3.5-2*a, 'v=10 cm/s',fontsize=fts)
ax99.text(3.1, 3.5-3*a, 'v= 5 cm/s',fontsize=fts)

ax99.hlines(3.5,-2,-1,color='tab:blue')
ax99.hlines(3.5-a,-2,-1,color='tab:orange')
ax99.hlines(3.5-2*a,-2,-1,color='tab:green')
ax99.text(-0.9, 3.5, 'D=7.5mm',fontsize=fts)
ax99.text(-0.9, 3.5-a, 'D=15mm',fontsize=fts)
ax99.text(-0.9, 3.5-2*a, 'D=25mm',fontsize=fts)



ax99.hlines(0,3.9,4.1,color='black')
ax99.text(2.7,0,'X=0m',fontsize=fts)

format_axes(ax99,'Distance Z (cm)','Position X (cm)', [0.02,0.95,'a2.',10], "upper right")


#############################################################################
#############################################################################

with cbook.get_sample_data('LOOMING_PAPER\\1-Fluid_Dynamics\\2-Plots\\sphere11.png') as image_file:
    image = plt.imread(image_file)

ax1.imshow(image)
ax1.axis('off') 
format_axes(ax1,'','', [0.02,0.95,'b1.',10], "upper right")
ax1.annotate("",xy=(150, 600), xycoords='data',xytext=(50, 600), textcoords='data',arrowprops=dict(arrowstyle="->"))
ax1.annotate("",xy=(50, 500), xycoords='data',xytext=(50, 600), textcoords='data',arrowprops=dict(arrowstyle="->"))
ax1.annotate("",xy=(20, 630), xycoords='data',xytext=(50, 600), textcoords='data',arrowprops=dict(arrowstyle="->"))
ax1.text(50,500,"x")
ax1.text(150,600,"z")
ax1.text(5,620,"y")



rect = patches.Rectangle((650, 250), 100, 100, linewidth=1, edgecolor='tab:pink', facecolor='none')
ax1.add_patch(rect)
rect = patches.Rectangle((650, 250), 100, 100, linewidth=1, edgecolor='tab:pink', facecolor='none')
ax2.add_patch(rect)
#############################################################################
#############################################################################

with cbook.get_sample_data('LOOMING_PAPER\\1-Fluid_Dynamics\\2-Plots\\sphere2.png') as image_file:
    image = plt.imread(image_file)

ax2.imshow(image)
ax2.axis('off') 
format_axes(ax2,'','', [0.02,0.95,'b2.',10], "upper right")
ax2.annotate("",xy=(150, 600), xycoords='data',xytext=(50, 600), textcoords='data',arrowprops=dict(arrowstyle="->"))
ax2.annotate("",xy=(50, 500), xycoords='data',xytext=(50, 600), textcoords='data',arrowprops=dict(arrowstyle="->"))
ax2.annotate("",xy=(20, 630), xycoords='data',xytext=(50, 600), textcoords='data',arrowprops=dict(arrowstyle="->"))
ax2.text(50,500,"x")
ax2.text(150,600,"z")
ax2.text(5,620,"y")

circle = plt.Circle((567, 305), 78, color='tab:green', clip_on=False, fill=False)
ax2.add_patch(circle)
#############################################################################
#############################################################################

radius=[7.5, 15, 25]
velocities=[5,10,20,40]

staMomentum=[]
midMomentum=[]
maxMomentum=[]


NORM=False

for v in velocities:
    for r in radius:
        T,V=extract_momentum(v,r)
        
        staMomentum+=[V[3]]
        midMomentum+=[V[int(len(V)/2)]]
        maxMomentum+=[max(V)]

staMomentum=np.asarray(staMomentum).reshape((4,-1))
midMomentum=np.asarray(midMomentum).reshape((4,-1))
maxMomentum=np.asarray(maxMomentum).reshape((4,-1))
"""
if NORM:
    staMomentum=staMomentum/np.max(maxMomentum)
    midMomentum=midMomentum/np.max(maxMomentum)
    maxMomentum=maxMomentum/np.max(maxMomentum)
"""

#imshow3d(ax,staMomentum, pos=0,norm='custom',cmap='seismic')
#imshow3d(ax,midMomentum, pos=1,norm='custom',cmap='seismic')
#imshow3d(ax,maxMomentum, pos=2,norm='custom',cmap='seismic')


imshow3d(ax5,staMomentum, pos=0,norm='custom',cmap='YlOrRd')
imshow3d(ax5,midMomentum, pos=1,norm='custom',cmap='YlOrRd')
imshow3d(ax5,maxMomentum, pos=2,norm='custom',cmap='YlOrRd')





ax5.xaxis.set_major_locator(ticker.FixedLocator(([0.5,1.5,2.5])))
ax5.xaxis.set_major_formatter(ticker.FixedFormatter((["7.5","15","25"])))

ax5.yaxis.set_major_locator(ticker.FixedLocator(([0.5,1.5,2.5,3.5])))
ax5.yaxis.set_major_formatter(ticker.FixedFormatter((["5","10","20","40"])))

ax5.zaxis.set_major_locator(ticker.FixedLocator(([0,1,2])))
ax5.zaxis.set_major_formatter(ticker.FixedFormatter((["1%","50%","100%"])))


ax5.set(xlabel="Diameter (mm)", ylabel="Velocity (cm/s)", zlabel="Normalized time")
ax5.grid(False)
#ax.set_xticks([])
#ax.set_yticks([])
#ax.set_zticks([])
ax5.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax5.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax5.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
#ax.xaxis.line.set_color((1.0, 1.0, 1.0, 0.0))
#ax.yaxis.line.set_color((1.0, 1.0, 1.0, 0.0))
#ax.zaxis.line.set_color((1.0, 1.0, 1.0, 0.0))

ax5.view_init(20, -40)

tmp_planes = ax5.zaxis._PLANES 
ax5.zaxis._PLANES = ( tmp_planes[2], tmp_planes[3], 
                     tmp_planes[0], tmp_planes[1], 
                     tmp_planes[4], tmp_planes[5])


#############################################################################
#############################################################################
radius=[7.5, 15, 25]
velocities=[5,10,20,40]

list_velo=[]
for r in radius:
    for v in velocities:
        T,V=extract_momentum(v,r)
        list_velo+=[max(V)]
    ax6.plot(velocities,list_velo,'.')
    list_velo=[]
format_axes(ax6,'Velocity (cm/s)','Maximum Momentum (N.s)', [0.02,0.95,'c2.',10], "upper center")        


draw_all_curves(ax4)
ax4.view_init(10, -20)    
#ax.grid(False)
ax4.set_ylim([-0.8,0])    


ab = AnnotationBbox(getImage('1-Fluid_Dynamics\\2-Plots\\sphere_zoom_A.png',0.15), (600, 600), frameon=True,bboxprops = 
                            dict(edgecolor='tab:pink'))
ax1.add_artist(ab)

ab = AnnotationBbox(getImage('1-Fluid_Dynamics\\2-Plots\\sphere_zoom_B.png',0.15), (600, 600), frameon=True,bboxprops = 
                            dict(edgecolor='tab:pink'))
ax2.add_artist(ab)




ax4.get_proj = lambda: np.dot(Axes3D.get_proj(ax4), np.diag([2, 1.5,1, 1]))




cbar = ax3.figure.colorbar(ax3.imshow(np.linspace(0,0.2,12).reshape(4,-1), cmap='bwr'), ax=ax3, label='Velocity (m.s)')
cbar = ax3.figure.colorbar(ax3.imshow(maxMomentum, cmap='YlOrRd'), ax=ax3, label='Momentum (N.s)')
ax3.imshow(np.zeros(12).reshape(4,-1),cmap='Greys')


arrow_prop_dict = dict(mutation_scale=20, arrowstyle='->', color='tab:pink', shrinkA=0, shrinkB=0,linewidth=3)
a = Arrow3D([20, 20], [-0.2, -0.2], [3, 0], **arrow_prop_dict)
ax4.add_artist(a)
a = Arrow3D([20, 20], [-0.002, -0.002], [11, 8], **arrow_prop_dict)
ax4.add_artist(a)

ax4.text2D(-0.35, 0.95, "b3.", transform=ax4.transAxes, fontweight="bold", fontsize=10)
ax.text2D(-0.05, 0.9, "a1.", transform=ax.transAxes, fontweight="bold", fontsize=10)
ax5.text2D(0, 0.95, "c1.", transform=ax5.transAxes, fontweight="bold", fontsize=10)


fig.tight_layout()
#plt.subplots_adjust(wspace=0.3,hspace=0.3)
pp.savefig(fig)
pp.close()

In [ ]:
SmallDF